In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Dataset folder
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import datasets,models,layers
from tensorflow.keras.utils import to_categorical

DATASET_PATH = '/kaggle/input/mais-202-winter-2022/'


lb_to_str_csv = DATASET_PATH + 'label_int_to_str_mapping.csv'
train_lb_csv = DATASET_PATH + 'train_labels.csv'

lb_to_str_df = pd.read_csv(lb_to_str_csv)
lb_to_str_df.head()
train_lb_df = pd.read_csv(train_lb_csv)
train_lb_df.head()

In [ ]:
train_images = np.load(DATASET_PATH+"train_images.npy")
def show_image(arr):
    two_d = (np.reshape(arr, (28, 28)) * 255).astype(np.uint8)
    plt.imshow(two_d, interpolation='nearest')
    plt.show()

show_image(train_images[0]) # 0 is the index of the training image you want to display
####################
TRAIN_NPY = DATASET_PATH + 'train_images.npy'
TEST_NPY = DATASET_PATH + 'test_images.npy'
train_images = np.load(TRAIN_NPY)
test_images = np.load(TEST_NPY)
# print(train_images[0])
show_img = np.reshape(train_images[5], (28,28))
train_images = np.reshape(train_images, (-1, 28,28,1)) / 255
test_images = np.reshape(test_images, (-1, 28, 28,1)) / 255
plt.imshow(show_img)
plt.show()

In [ ]:
from tensorflow.keras.utils import to_categorical
train_size = 40000

labels = train_lb_df['label'].tolist()
x_train = train_images[:train_size]
y_train = np.array(labels)[:train_size]

x_val = train_images[train_size:]
y_val = np.array(labels)[train_size:]


y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

In [ ]:
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
batch_size = 256
num_classes = 10
epochs = 20

#input image dimensions
img_rows, img_cols = 28, 28

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
y_test = model.predict(test_images)
y_test = np.argmax(y_test, axis=1)

df_test = pd.read_csv(DATASET_PATH+'sample_submission.csv')
df_test['label'] = y_test
df_test.to_csv('submission.csv', index=False)
